<a href="https://colab.research.google.com/github/radza99/Lab/blob/main/SentimentAnalysisBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ใช้ DecisionTreeClassifier และ เปลี่ยน data set**

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [62]:
# โหลดข้อมูล
df = pd.read_csv("my_sentiment_data.csv")

In [64]:
df.head(30)

,text,sentiment
0,วันนี้อากาศสดใส เหมาะกับการออกกำลังกาย,positive
1,ฉันรู้สึกดีใจที่ได้พบเพื่อนเก่า,positive
2,อาหารร้านนี้รสชาติธรรมดา ไม่ได้พิเศษ,neutral
3,บริการช้ามาก รอนานเกินไป,negative
4,สินค้าคุณภาพดี ใช้งานได้ดีมาก,positive
5,ไม่พอใจกับการส่งสินค้าที่ล่าช้า,negative
6,งานนี้สนุกและได้เรียนรู้เยอะมาก,positive
7,รู้สึกเหนื่อยและเครียดกับงานมาก,negative
8,เพื่อนร่วมงานช่วยเหลือดีมาก,positive
9,สถานที่สะอาดและเป็นระเบียบดี,positive


In [65]:
# โหลด tokenizer และ model ของ WangchanBERTa
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
model = AutoModel.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

In [66]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [67]:
X = df["text"].apply(get_embedding).tolist()
X = torch.tensor(X)
y = df["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y, test_size=0.2, random_state=42)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `CamembertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y, test_size=0.2, random_state=42)


In [69]:
# ✅ ใช้ Decision Tree แทน Logistic Regression
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [70]:
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6666666666666666

Classification Report:
               precision    recall  f1-score   support

    negative       0.50      1.00      0.67         2
    positive       1.00      0.50      0.67         4

    accuracy                           0.67         6
   macro avg       0.75      0.75      0.67         6
weighted avg       0.83      0.67      0.67         6



In [73]:
new_text = ["ระบบแอปพลิเคชันใช้งานยากและช้า	"]
inputs = tokenizer(new_text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    new_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy().reshape(1, -1)

predicted_sentiment = classifier.predict(new_embedding)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `CamembertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [74]:

print("\nNew Text Prediction:")
print("Text:", new_text[0])
print("Predicted Sentiment:", predicted_sentiment[0])


New Text Prediction:
Text: ระบบแอปพลิเคชันใช้งานยากและช้า	
Predicted Sentiment: negative
